In [1]:
import skimage

In [2]:
# read image
img = skimage.io.imread('/data/vision/polina/users/clintonw/code/kubric/output/0/rgba_00064.png')

In [4]:
import logging
import pdb
import shutil
import sys, os
sys.path.extend(['/usr/lib/x86_64-linux-gnu', '/data/vision/polina/users/clintonw/anaconda3/envs/cuda11/lib'])
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu:/data/vision/polina/users/clintonw/anaconda3/envs/cuda11/lib'
import kubric as kb
from kubric.simulator import PyBullet
from kubric.renderer import Blender
import numpy as np
import os.path as osp
import pybullet as pb
import bpy
import argparse
# --- Some configuration values
# the region in which to place objects [(min), (max)]
SPAWN_REGION = [(-6, -6, .3), (6, 6, 5)]
CLEVR_OBJECTS = ("cube", "cylinder", "sphere", "cone", "torus")
KUBASIC_OBJECTS = ("cube", "cylinder", "sphere", "cone", "torus", "gear",
                   "torus_knot", "sponge", "spot", "teapot", "suzanne")

OUT_DIR = "./0"
TMP_DIR = osp.expandvars("$HOME/tmp")
# # --- CLI arguments
FLAGS = argparse.Namespace(
frame_rate=24, step_rate=240, frame_start=1, logging_level="INFO",
seed=None, resolution="256", scratch_dir=TMP_DIR, job_dir=OUT_DIR,
    objects_set="clevr",
    min_num_objects=3,
    max_num_objects=7,
    floor_friction=0.3,
    floor_restitution=0.5,
    backgrounds_split="train",
    camera="spiral",
    start_id=0,
    kubasic_assets=osp.expandvars("$SCRATCH/datasets/kubric-public/assets/KuBasic/KuBasic.json"),
    hdri_assets=osp.expandvars("$SCRATCH/datasets/kubric-public/assets/HDRI_haven/HDRI_haven.json"),
    gso_assets=osp.expandvars("$SCRATCH/datasets/kubric-public/assets/GSO/GSO.json"),
    save_state=False,
    frame_end=256,
    num_trajectories=1,
    overwrite=False,
)
base_dir = FLAGS.job_dir
pb_client = pb.connect(pb.DIRECT)
kubasic = kb.AssetSource.from_manifest(FLAGS.kubasic_assets)
hdri_source = kb.AssetSource.from_manifest(FLAGS.hdri_assets)
train_backgrounds, test_backgrounds = hdri_source.get_test_split(fraction=0.1)


In [5]:
for i in range(FLAGS.start_id, FLAGS.num_trajectories + FLAGS.start_id):
    # --- Common setups & resources
    FLAGS.job_dir = f"{base_dir}/{i}"
    if osp.exists(FLAGS.job_dir):
        if FLAGS.overwrite:
            shutil.rmtree(FLAGS.job_dir)
        else:
            continue

    scene, rng, output_dir, scratch_dir = kb.setup(FLAGS)
    simulator = PyBullet(scene, scratch_dir, client=pb_client)
    renderer = Blender(scene, scratch_dir, samples_per_pixel=64)
    resolution = scene.resolution

    # --- Populate the scene
    # background HDRI
    if FLAGS.backgrounds_split == "train":
        hdri_id = rng.choice(train_backgrounds)
    else:
        hdri_id = rng.choice(test_backgrounds)
    background_hdri = hdri_source.create(asset_id=hdri_id)
    #assert isinstance(background_hdri, kb.Texture)
    logging.info("Using background %s", hdri_id)
    scene.metadata["background"] = hdri_id
    renderer._set_ambient_light_hdri(background_hdri.filename)

    # Dome
    dome = kubasic.create(asset_id="dome", name="dome",
                                                friction=FLAGS.floor_friction,
                                                restitution=FLAGS.floor_restitution,
                                                static=True, background=True)
    assert isinstance(dome, kb.FileBasedObject)
    scene += dome
    dome_blender = dome.linked_objects[renderer]
    texture_node = dome_blender.data.materials[0].node_tree.nodes["Image Texture"]
    texture_node.image = bpy.data.images.load(background_hdri.filename)

    # Camera
    logging.info("Setting up the Camera...")
    focal_length = 30. # focal length (mm)
    sensor_width = 32
    scene.camera = kb.PerspectiveCamera(focal_length=focal_length, sensor_width=sensor_width)

    frame_list = []
    num_frames = (FLAGS.frame_end + 1) - (FLAGS.frame_start)
    if FLAGS.camera == "spiral":
        rotations = 6
        R = 15 + np.random.randn(num_frames)
        phis = np.linspace(0, rotations*2*np.pi, num_frames)
        thetas = np.linspace(np.pi*.5, np.pi*.08, num_frames)
        positions = np.stack([R * np.sin(thetas) * np.cos(phis),
                                                    R * np.sin(thetas) * np.sin(phis),
                                                    R * np.cos(thetas) + .1], axis=1)
        for frame in range(FLAGS.frame_start, FLAGS.frame_end + 1):
            ix = frame - (FLAGS.frame_start)
            scene.camera.position = positions[ix]
            scene.camera.look_at((np.random.randn()*.2, np.random.randn()*.2, .7 + np.random.randn()*.2))
            scene.camera.keyframe_insert("position", frame)
            scene.camera.keyframe_insert("quaternion", frame)
            frame_data = {
                "file_path": "rgba_{:04d}.png".format(ix),
                "transform_matrix": scene.camera.matrix_world.tolist(),
            }
            frame_list.append(frame_data)
        
    elif FLAGS.camera == "random":    # Random position in half-sphere-shell
        for frame in range(FLAGS.frame_start, FLAGS.frame_end + 1):
            ix = frame - (FLAGS.frame_start)
            scene.camera.position = kb.sample_point_in_half_sphere_shell(
                    inner_radius=8., outer_radius=10., offset=0.1)
            scene.camera.look_at((np.random.randn()*.2, np.random.randn()*.2, .7 + np.random.randn()*.2))
            scene.camera.keyframe_insert("position", frame)
            scene.camera.keyframe_insert("quaternion", frame)
            frame_data = {
                "file_path": "rgba_{:04d}.png".format(ix),
                "transform_matrix": scene.camera.matrix_world.tolist(),
            }
            frame_list.append(frame_data)


    # Add random objects
    num_objects = rng.randint(FLAGS.min_num_objects,
                                                        FLAGS.max_num_objects+1)
    logging.info("Randomly placing %d objects:", num_objects)
    for i in range(num_objects):
        if FLAGS.objects_set == "clevr":
            shape_name = rng.choice(CLEVR_OBJECTS)
            size_label, size = kb.randomness.sample_sizes("clevr", rng)
            color_label, random_color = kb.randomness.sample_color("clevr", rng)
        else:    # FLAGS.object_set == "kubasic":
            shape_name = rng.choice(KUBASIC_OBJECTS)
            size_label, size = kb.randomness.sample_sizes("uniform", rng)
            color_label, random_color = kb.randomness.sample_color("uniform_hue", rng)

        material_name = rng.choice(["metal", "rubber"])
        obj = kubasic.create(
                asset_id=shape_name, scale=size,
                name=f"{size_label} {color_label} {material_name} {shape_name}")
        assert isinstance(obj, kb.FileBasedObject)

        if material_name == "metal":
            obj.material = kb.PrincipledBSDFMaterial(color=random_color, metallic=1.0,
                                                                                            roughness=0.2, ior=2.5)
            obj.friction = 0.4
            obj.restitution = 0.3
            obj.mass *= 2.7 * size**3
        else:    # material_name == "rubber"
            obj.material = kb.PrincipledBSDFMaterial(color=random_color, metallic=0.,
                                                                                            ior=1.25, roughness=0.7,
                                                                                            specular=0.33)
            obj.friction = 0.8
            obj.restitution = 0.7
            obj.mass *= 1.1 * size**3

        obj.metadata = {
                "shape": shape_name.lower(),
                "size": size,
                "size_label": size_label,
                "material": material_name.lower(),
                "color": random_color.rgb,
                "color_label": color_label,
        }
        scene.add(obj)
        kb.move_until_no_overlap(obj, simulator, spawn_region=SPAWN_REGION, rng=rng)
        obj.velocity = (0,0,0)

        logging.info("        Added %s at %s", obj.asset_id, obj.position)

    logging.info("Rendering the scene ...")
    # layers = ['rgba', 'depth', 'segmentation', 'normal', 'object_coordinates']
    layers = ['rgba', 'segmentation']
    data_stack = renderer.render(return_layers=layers)

    # --- Postprocessing
    kb.compute_visibility(data_stack["segmentation"], scene.assets)
    visible_foreground_assets = [asset for asset in scene.foreground_assets
                                                            if np.max(asset.metadata["visibility"]) > 0]
    visible_foreground_assets = sorted(    # sort assets by their visibility
        visible_foreground_assets,
        key=lambda asset: np.sum(asset.metadata["visibility"]),
        reverse=True)

    data_stack["segmentation"] = kb.adjust_segmentation_idxs(
        data_stack["segmentation"],
        scene.assets,
        visible_foreground_assets)
    scene.metadata["num_instances"] = len(visible_foreground_assets)

    # Save to image files
    kb.write_image_dict(data_stack, output_dir)
    kb.post_processing.compute_bboxes(data_stack["segmentation"],
                                                                        visible_foreground_assets)

    # nerfstudio format
    if True:
        kb.write_json(filename=output_dir / "transforms.json", data={
            "fl_x": focal_length * resolution[0] / sensor_width,
            "fl_y": focal_length * resolution[1] / sensor_width,
            "cx": resolution[0]/2,
            "cy": resolution[1]/2,
            "h": resolution[0],
            "w": resolution[1],
            "k1": 0.0,
            "aabb_scale": 8,
            "camera_angle_x": scene.camera.field_of_view,
            "frames": frame_list,
        })

    # nerf / instant ngp format
    else:
        kb.write_json(filename=output_dir / "transforms.json", data={
            "aabb_scale": 2,
            "scale": 0.18,
            "offset": [0.5, 0.5, 0.5],
            "camera_angle_x": scene.camera.field_of_view,
            "frames": frame_list,
        })
        
    # --- Metadata
    logging.info("Collecting and storing metadata for each object.")
    kb.write_json(filename=output_dir / "metadata.json", data={
            "flags": vars(FLAGS),
            "metadata": kb.get_scene_metadata(scene),
            "camera": kb.get_camera_info(scene.camera),
            "instances": kb.get_instance_info(scene, visible_foreground_assets),
    })

    shutil.rmtree(scratch_dir)
    kb.done()
try:
    pb.disconnect()
except Exception:    # pylint: disable=broad-except
    pass    # cleanup is already done


2023-01-12 17:48:47.843342: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Fra:1 Mem:37.51M (Peak 293.54M) | Time:00:00.00 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | dome
Fra:1 Mem:37.51M (Peak 293.54M) | Time:00:00.00 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | large yellow metal cone
Fra:1 Mem:44.29M (Peak 293.54M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Initializing
Fra:1 Mem:34.71M (Peak 293.54M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Waiting for render to start
Fra:1 Mem:34.71M (Peak 293.54M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Loading render kernels (may take a few minutes the first time)
Fra:1 Mem:34.71M (Peak 293.54M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Updating Scene
Fra:1 Mem:34.71M (Peak 293.54M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Updating Shaders
Fra:1 Mem:35.22M (Peak 293.54M) | Time:00:00.02 | Mem:0.01M, Peak:0.01M | Scene, ViewLayer | Updating Procedurals
Fra:1 Mem:35.22M (Peak 2